In [ ]:
from signalflow.data import BinanceSpotLoader
from pathlib import Path

In [ ]:
loader = BinanceSpotLoader(db_path=Path("raw_data.duckdb"))

await loader.download(
    pairs=["BTCUSDT", "ETHUSDT", "SOLUSDT", "XRPUSDT", "BNBUSDT"],
    days=100,
    fill_gaps=False
)